In [1]:
1+1

2

## 1. FAISS 설치

먼저 FAISS를 설치해야 합니다. CPU 버전과 GPU 버전이 있습니다:[3][4]

**CPU 버전 설치:**
```bash
pip install faiss-cpu
```

**GPU 버전 설치 (CUDA 필요):**
```bash
pip install faiss-gpu
```

**설치 확인:**

In [2]:
import faiss

print(faiss.__version__)

1.12.0


In [3]:
import numpy as np
import faiss

# 벡터 차원 설정
d = 64  # 벡터 차원
nb = 100000  # 데이터베이스 벡터 개수
nq = 10000  # 쿼리 벡터 개수

# 시드 설정 (재현 가능한 결과를 위해)
np.random.seed(1234)

# 데이터베이스 벡터 생성 (검색 대상)
xb = np.random.random((nb, d)).astype("float32")
xb[:, 0] += np.arange(nb) / 1000.0

# 쿼리 벡터 생성 (검색할 벡터들)
xq = np.random.random((nq, d)).astype("float32")
xq[:, 0] += np.arange(nq) / 1000.0

In [4]:
# 인덱스 생성
index = faiss.IndexFlatL2(d)

# 인덱스 상태 확인
print(f"훈련 여부: {index.is_trained}")  # True

# 벡터를 인덱스에 추가
index.add(xb)
print(f"인덱스된 벡터 수: {index.ntotal}")  # 100000

# 검색 수행
k = 4  # 상위 4개 유사 벡터 찾기
D, I = index.search(xq, k)

print("처음 5개 쿼리의 가장 유사한 4개 벡터 ID:")
print(I[:5])
print("해당 거리값:")
print(D[:5])

훈련 여부: True
인덱스된 벡터 수: 100000
처음 5개 쿼리의 가장 유사한 4개 벡터 ID:
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
해당 거리값:
[[6.815506  6.8894653 7.3956795 7.4290257]
 [6.6041145 6.679699  6.7209625 6.828682 ]
 [6.4703827 6.8578606 7.0043755 7.036564 ]
 [5.573681  6.4075394 7.1395187 7.355591 ]
 [5.409401  6.232216  6.4173393 6.5743713]]


In [5]:
# 사용자 정의 ID 배열 생성
ids = np.array([100, 99, 98, 97, 96, 95, 94, 93, 92, 91], dtype="int64")
vector_sample = np.random.random((10, d)).astype("float32")

# ID 매핑이 가능한 인덱스 생성
index = faiss.IndexFlatL2(d)
index = faiss.IndexIDMap2(index)

# ID와 함께 벡터 추가
index.add_with_ids(vector_sample, ids)

# ID를 사용한 검색
query_vector = np.random.random((1, d)).astype("float32")
D, I = index.search(query_vector, 3)
print(f"검색된 벡터 ID: {I[0]}")  # 실제 ID 값들 출력

# ID로 원본 벡터 복원
original_vector = index.reconstruct(100)  # ID 100의 벡터 복원
print(f"복원된 벡터 차원: {original_vector.shape}")

검색된 벡터 ID: [94 98 93]
복원된 벡터 차원: (64,)


In [6]:
# 인덱스 저장
faiss.write_index(index, "my_index.faiss")

# 인덱스 로드
loaded_index = faiss.read_index("my_index.faiss")

# 로드된 인덱스로 검색
D, I = loaded_index.search(query_vector, 3)
print(f"로드된 인덱스 검색 결과: {I[0]}")

로드된 인덱스 검색 결과: [94 98 93]


In [7]:
# 내적(Inner Product) 기반 유사도 검색
index_ip = faiss.IndexFlatIP(d)
index_ip.add(xb)
D, I = index_ip.search(xq, k)

In [8]:
# 더 빠른 검색을 위한 클러스터링 기반 인덱스
nlist = 100  # 클러스터 수
quantizer = faiss.IndexFlatL2(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)

# 훈련 필요
index_ivf.train(xb)
index_ivf.add(xb)

# 검색 매개변수 설정
index_ivf.nprobe = 10  # 검색할 클러스터 수
D, I = index_ivf.search(xq, k)

In [9]:
# 가상의 텍스트 임베딩 데이터
documents = [
    "machine learning tutorial",
    "deep learning basics",
    "python programming",
    "data science guide",
    "artificial intelligence",
]

# 임베딩 벡터 (실제로는 사전 훈련된 모델 사용)
embeddings = np.random.random((len(documents), 128)).astype("float32")
doc_ids = np.arange(len(documents)).astype("int64")

# 인덱스 생성 및 데이터 추가
d = 128
index = faiss.IndexFlatL2(d)
index = faiss.IndexIDMap2(index)
index.add_with_ids(embeddings, doc_ids)

# 쿼리 임베딩으로 검색
query_embedding = np.random.random((1, d)).astype("float32")
D, I = index.search(query_embedding, 3)

print("가장 유사한 문서들:")
for i, doc_id in enumerate(I[0]):
    print(f"{i+1}. {documents[doc_id]} (거리: {D[0][i]:.4f})")

가장 유사한 문서들:
1. machine learning tutorial (거리: 19.4335)
2. data science guide (거리: 20.4205)
3. deep learning basics (거리: 23.2298)
